In [1]:
import os
import pickle
import time
import streamlit as st
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredURLLoader 
from langchain import OpenAI
from langchain.vectorstores import FAISS

In [4]:
loader=UnstructuredURLLoader(urls=[
    'https://www.cbsl.gov.lk/en/financial-system/financial-markets/government-securities-market',
    'https://www.sc.com/ke/investments/learn/understanding-bonds-for-beginners/'
])
data=loader.load()
len(data)

2

In [5]:
data[0]

Document(metadata={'source': 'https://www.cbsl.gov.lk/en/financial-system/financial-markets/government-securities-market'}, page_content="Skip to main content\n\nEnglish\n\nසිංහල\n\nதமிழ்\n\nNavigation\n\nABOUT\n\nAbout the Bank\n\nOverview\n\nHistory\n\nVision, Mission & Values\n\nObjectives\n\nFunctions\n\nOrganisational Structure\n\nCorporate Structure\n\nPrincipal Officers\n\nDepartments\n\nBank Premises\n\nBank Headquarters\n\nRegional Offices\n\nLibrary & Information Center\n\nCentre For Banking Studies\n\nThe Economic History Museum of Sri Lanka\n\nExter Report\n\nMONETARY POLICY\n\nAbout Monetary Policy\n\nOverview\n\nEconomic and Price Stability\n\nMonetary Policy Framework\n\nInstruments & Implementation\n\nPolicy Rates and Open Market Operations\n\nStatutory Reserve Requirement\n\nOther Policy Instruments\n\nMonetary Policy Committee\n\nStakeholder Engagement Committee\n\nMonetary Policy Communication\n\nOverview\n\nMonetary Policy Announcement Dates\n\nMonetary Policy Advan

In [6]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs=text_splitter.split_documents(data)
len(docs)

30

In [15]:
docs[1]

Document(metadata={'source': 'https://www.cbsl.gov.lk/en/financial-system/financial-markets/government-securities-market'}, page_content='Monetary Policy Review\n\nMonetary Policy Review - No. 5 of 2024\n\nMonetary Policy Review - No. 4 of 2024\n\nMonetary Policy Review - No. 3 of 2024\n\nMonetary Policy Review - No. 2 of 2024\n\nMonetary Policy Review - No. 1 of 2024\n\nmore\n\nFINANCIAL SYSTEM\n\nFinancial System Stability\n\nOverview\n\nMajor Functions\n\nBanking Sector\n\nNon Bank Finance and Leasing Sector\n\nPrimary Dealers\n\nMicrofinance Sector\n\nMoney Broking Industry\n\nMacroprudential Surveillance\n\nSustainable Finance\n\nFinancial Markets\n\nFinancial Markets Overview\n\nInterbank Call money Market\n\nDomestic Foreign Exchange Market\n\nGovernment Securities Market\n\nCorporate Debt Securities Market\n\nEquity Market\n\nDevelopment Finance and Access to Finance\n\nFinancial Infrastructure\n\nPayment and Settlement Systems\n\nCredit Information\n\nLaws and Regulations\n\nI

In [19]:
# Open a file in write mode
with open("documents_output.txt", "w", encoding="utf-8") as file:
    for i, doc in enumerate(docs):
        file.write(f"Document {i+1}:\n")
        file.write(doc.page_content)  # Writing the page content
        file.write("\n\n")  # Add some spacing between documents


In [20]:
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the documents 
embeddings = model.encode([d.page_content for d in docs])



c:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [31]:
import faiss
#get the dimension of the embeddings
d=embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
print(f"Total sentences indexed: {index.ntotal}")

Total sentences indexed: 30


In [35]:
# Define a query sentence
query_sentence = "How to invest treasury bonds?"
query_embedding = model.encode([query_sentence])

# Perform the search
k = 1  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Display the results
print(f"Query: {query_sentence}")

print("Most similar sentences:")
for i, idx in enumerate(indices[0]):
    print(f"{i + 1}: {docs[idx].page_content} (Distance: {distances[0][i]})")

Query: How to invest treasury bonds?
Most similar sentences:
1: Thank you for visiting www.sc.com/ke

Proceed

Cultivate an Understanding of Bonds – Beginners Guide

Managing your wealth well is like tending a beautiful formal garden – you need to start with good soil and a good set of tools. Just as good soil has the proper fertility to nourish a plant, having the right foundation in financial literacy should empower you to potentially cultivate a successful investment portfolio. Cultivate an Understanding of Bonds is part of our financial education series to help educate you on the fundamentals of investing as you tend your very own financial garden.

What is a Bond?

If you are looking to build up a well-diversified portfolio, you will usually be advised to include both stocks and bonds among your investments. While stocks may offer you the potential for capital appreciation, bonds may provide a steady stream of investment income, and play an important role of potentially lowering y